In [34]:
#Library load
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [35]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

In [36]:
df.shape

(569, 33)

In [37]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [38]:
df.drop(columns=["id",'Unnamed: 32'],inplace=True)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [40]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [41]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [42]:
y_train.shape

(455,)

### Convert numpy array to Tensor

In [10]:
X_train_tensor=torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [11]:
X_train.shape

(455, 30)

### Defining The model

In [ ]:



class simpleNN:

    def __init__(self,X):
        self.weight=torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
        self.bias=torch.zeros(1,dtype=torch.float64,requires_grad=True)

    def forward(self,X):    
        z=torch.matmul(X,self.weight) +self.bias
        y_pred=torch.sigmoid(z)
        return y_pred  

    def loss_function(self, y_pred, y):
        # Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

        # Calculate loss
        loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
        return loss
        

    


In [35]:
#important parameter
epochs=10
learning_rate=0.1

### Training Pipeline

In [38]:
model=simpleNN(X_train_tensor)

for epoch in range(epochs):
    #forward pass
    y_pred=model.forward(X_train_tensor)
  

    # loss calculate
    loss = model.loss_function(y_pred, y_train_tensor)

    # backward pass
    loss.backward()

  # parameters update
    with torch.no_grad():
        model.weight -= learning_rate * model.weight.grad
        model.bias -= learning_rate * model.bias.grad
        

  # zero gradients
    model.weight.grad.zero_()
    model.bias.grad.zero_()

  # print loss in each epoch
    print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.5231049740700784
Epoch: 2, Loss: 3.3958938996611736
Epoch: 3, Loss: 3.2670150209802866
Epoch: 4, Loss: 3.139271177202119
Epoch: 5, Loss: 3.009845146896282
Epoch: 6, Loss: 2.8805397052698933
Epoch: 7, Loss: 2.7410114027833496
Epoch: 8, Loss: 2.5996927649424264
Epoch: 9, Loss: 2.458910954189579
Epoch: 10, Loss: 2.3177969290709486


In [39]:
model.bias

tensor([-0.0226], dtype=torch.float64, requires_grad=True)

In [40]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5654047131538391


torch.float32

In [48]:
## Using Pytorch's build in layer,optimizer(optim),built in loss function

##As x_train,y_train both must be same dtype(here we use float,previously itwas x_train float,y_train int)
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [50]:
print(y_train.dtype)
print(y_train_tensor.dtype)


int32
torch.float32


In [22]:
# create the model class
import torch.nn as nn

class MySimpleNN(nn.Module):
    def __init__(self,n_features):
        super().__init__()
        self.linear=nn.Linear(n_features,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self,features):
        out=self.linear(features)
        out=self.sigmoid(out)
        return out


In [23]:
# Create parameter
epochs=10
learning_rate=0.1

In [51]:
# define loss function

loss_function=nn.BCELoss()

In [52]:
# create model object
model=MySimpleNN(X_train_tensor.shape[1])

In [53]:
# define optimizer
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate) #model.parameter is a iterator

In [55]:
# training 

for epoch in range(epochs):

    y_pred=model(X_train_tensor) #call forward pass

    loss=loss_function(y_pred,y_train_tensor.view(-1,1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"epoch: {epoch+1}; loss: {loss}")
    



epoch: 1; loss: 0.7018051147460938
epoch: 2; loss: 0.5406448245048523
epoch: 3; loss: 0.4546789228916168
epoch: 4; loss: 0.4010370671749115
epoch: 5; loss: 0.36343908309936523
epoch: 6; loss: 0.33510977029800415
epoch: 7; loss: 0.31272268295288086
epoch: 8; loss: 0.294427752494812
epoch: 9; loss: 0.2791002094745636
epoch: 10; loss: 0.2660098969936371


In [56]:

# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5344721674919128
